In [ ]:
import pandas as pd
from pathlib import Path 

In [ ]:
fn = sorted(Path('.').glob('recipes-2*.jl'))[-1]

In [ ]:
recipes_df = (
    pd.read_json(fn, lines=True)
    .drop_duplicates('id')
    .set_index('id')
    .rename(
        columns={
            'total': 'time_total',
            'prep': 'time_prep',
            'cook': 'time_cook',
            'active': 'time_active',
            'inactive': 'time_inactive',
        },
    )
)
recipes_df['level'] = recipes_df['level'].astype('category')
# TODO: replace time_... values from str to timedelta()
del recipes_df['']
recipes_df = recipes_df[
    [c for c in recipes_df.columns if 'time' not in c]
    + [c for c in recipes_df.columns if 'time' in c]
]
for col in recipes_df.columns:
    if 'time' in col:
        recipes_df[col] = pd.to_timedelta(recipes_df[col], errors='coerce')
recipes_df

In [ ]:
recipes_df.time_total.sort_values().reset_index()

In [ ]:
import re
s1aa = r"[\d\ \/\.]*\d"
s1a = fr"(Approximately )?({s1aa} to )?({s1aa})\-?"
s1b = r"(pinch|grind|teaspoon|tablespoon|cup|quart|gallon|ounce|pound|gram|kilogram|ml)s?"
s1c = r"(plus|minus)"
s1 = fr"{s1a}( {s1b})?( {s1c} {s1a}( {s1b})?)?"
s2 = r"\(.*\)"
s3 = r"(all-purpose, unbleached flour|[^,^(]*)"
s4 = r"\, appr[^,^(]*"
s5 = r"\,[^(]*"
s6 = r"\(.*"
s = fr"^(?P<amount>{s1} )?(?P<alt_amount>{s2} )?(?P<thing>{s3})(?P<alt_amount2>{s4})?(?P<action>{s5})?(?P<note>{s6})?$"
ing = re.compile(s)
# for ingredients_list in recipes_df.ingredients:
#     for ingredient in ingredients_list:
# #         print(ingredient)
#         if ingredient.endswith(':'):
#             break
#         s = ing.match(ingredient)['thing']
#         if s is not None:
#             print(s)
            
pd.Series(
    [ing.match(ingredient)['thing'] for ingredients_list in recipes_df.ingredients for ingredient in ingredients_list if not ingredient.endswith(':')]
).value_counts().head(30)